In [1]:
''' 
Link to Documentation: https://github.com/ultralytics/yolov5

RUN IN TERMINAL BEFORE PROCEEDED IN THE 'fashion-forecast' DIRECTORY:
>   pip install ultralytics
>   git clone https://github.com/ultralytics/yolov5  
>   cd yolov5
>   pip install -r requirements.txt  
>   cd ..
'''

" \nLink to Documentation: https://github.com/ultralytics/yolov5\n\nRUN IN TERMINAL BEFORE PROCEEDED IN THE 'fashion-forecast' DIRECTORY:\n>   pip install ultralytics\n>   git clone https://github.com/ultralytics/yolov5  \n>   cd yolov5\n>   pip install -r requirements.txt  \n>   cd ..\n"

In [2]:
import torch
import numpy as np
import pandas as pd
import os
from PIL import Image

%matplotlib inline 
from matplotlib import pyplot as plt

In [3]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True) #force_reload=True

Using cache found in C:\Users\cardon/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-16 Python-3.8.5 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [4]:
'''
CODE TO CHANGE DEVICE:
model.cpu()  # CPU
model.cuda()  # GPU
model.to(device)  # i.e. device=torch.device(0)
'''

'\nCODE TO CHANGE DEVICE:\nmodel.cpu()  # CPU\nmodel.cuda()  # GPU\nmodel.to(device)  # i.e. device=torch.device(0)\n'

In [25]:
img_list = []
img_names = []
directory = '../unpack-data/'
printout_size = 1000
seasons_only_csv = pd.read_csv('filtered_style_stats.csv')
counter = 0
for img_name in os.listdir(directory):
    if seasons_only_csv.iloc[counter, 0] != img_name:
        continue
    if counter%printout_size == 0:
        print(f"Running... {(counter/len(seasons_only_csv.index))*100:.2f}% finished")
    img_path = os.path.join(directory, img_name)
    print(counter)
    pil_img = Image.open(img_path)
    np_img = np.array(pil_img)
    #plt.imshow(np_img)             # used to display the images
    #plt.show()
    img_list.append(np_img)
    img_names.append(str(img_path))
    counter += 1
print(f'Finished with a set of {len(img_list)} images')

Running... 0.00% finished
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


In [6]:
results = model(img_list)  # inference
results.save()   # cropped detections dictionary
#results.show()

Saved 10 images to runs\detect\exp


In [7]:
detection_df = results.pandas().xyxy
print(len(detection_df))
'''
for idx, df in enumerate(detection_df):
    person_df = df[df['name']=='person']
    print(person_df.index)
'''

10


"\nfor idx, df in enumerate(detection_df):\n    person_df = df[df['name']=='person']\n    print(person_df.index)\n"

In [8]:
"""Takes the current results and saves only the information from the maximum 'person' detections"""

cleaned_df_lst = []
for idx, df in enumerate(detection_df):
    person_df = df[df['name']=='person']
    if person_df.empty == True:
        continue
    if (len(person_df.index) == 1):
        row = person_df.iloc[0].tolist()
        row.append(idx)
        cleaned_df_lst.append(row)
        continue
    # multiple detections -> choose the highest confidence of person
    maxValueIndex = person_df['confidence'].idxmax()
    row = person_df.iloc[maxValueIndex].tolist()
    row.append(idx)
    cleaned_df_lst.append(row)

col_names = list(detection_df[0].columns) + ['df_idx']
cleaned_df = pd.DataFrame(cleaned_df_lst, columns = col_names)
print(len(cleaned_df.index))
#print(cleaned_df)


9


In [9]:
"""Takes the cleaned df data and crops each image to it's detected bounding box dimensions"""
directory = 'C:/Users/cardon/OneDrive - Rensselaer Polytechnic Institute/SEM8/RCOS/unpack-data'
save_dir = 'C:/Users/cardon/OneDrive - Rensselaer Polytechnic Institute/SEM8/RCOS/crop-data'
df_idx = 0
for idx, img_name in enumerate(os.listdir(directory)):
    if idx == 8:
        break
    if idx not in cleaned_df['df_idx'].values:
        continue

    xmin, ymin, xmax, ymax = cleaned_df.loc[df_idx][['xmin', 'ymin', 'xmax', 'ymax']]
    #print(int(xmin), int(ymin), int(xmax), int(ymax))
    df_idx += 1

    img_path = os.path.join(directory, img_name)
    #print(img_path)
    pil_img = Image.open(img_path)
    np_img = np.array(pil_img)
    #plt.imshow(np_img)             
    #plt.show()
    crop_img = np_img[int(ymin):int(ymax), int(xmin):int(xmax)]
    #plt.imshow(crop_img)             
    #plt.show()
    save_path = os.path.join(save_dir, (img_name+'.jpg'))
    save_im = Image.fromarray(crop_img)
    save_im.save(save_path)